Arbitary Value impuatation

It consists of replacing NAN by an arbitary value (any value according to user wish)

In [ ]:
import pandas as pd
df=pd.read_csv("titanic.csv",usecols=["Age","Fare","Survived"])
df.head()

In [ ]:
def impute_nan(df,variable):
  df[variable+"_zero"]=df[variable].fillna(0)
  df[variable+"_hundred"]=df[variable].fillna(100)



In [ ]:
impute_nan(df,"Age")
df.head()

Advantages
1) Easy to implement 
2) Captures the importance of missingness if there is one

Disadvantages
1) Distorts the original distribution of the variable
2) If missings is not important it may mask the predicitve power of the original variable distorting its distribution
3) Hard to decide which value to use .


# **How to handle categorical missing values**

Technique 1: Frequent category Imputation

In [ ]:
import pandas as pd
df=pd.read_csv("train_house.csv",usecols=["BsmtQual","FireplaceQu","GarageType","SalePrice"])
df.head(4)

,BsmtQual,FireplaceQu,GarageType,SalePrice
0,Gd,NaN,Attchd,208500
1,Gd,TA,Attchd,181500
2,Gd,TA,Attchd,223500
3,TA,Gd,Detchd,140000


In [ ]:
df.isnull().mean().sort_values(ascending=True)

SalePrice      0.000000
BsmtQual       0.025342
GarageType     0.055479
FireplaceQu    0.472603
dtype: float64

First compute(Check) the frequency with every feature

In [ ]:
# Both method will give same result 
#1) method 1
df["BsmtQual"].value_counts()
#2) method 2
df.groupby(["BsmtQual"])["BsmtQual"].count()

TA    649
Gd    618
Ex    121
Fa     35
Name: BsmtQual, dtype: int64

In [ ]:
df["GarageType"].value_counts()

Attchd     870
Detchd     387
BuiltIn     88
Basment     19
CarPort      9
2Types       6
Name: GarageType, dtype: int64

In [ ]:
df["FireplaceQu"].value_counts()

Gd    380
TA    313
Fa     33
Ex     24
Po     20
Name: FireplaceQu, dtype: int64

Now replacing the NAN with most occured feature ex: Gd in FireplaceQu

First we need to find the category name using index otherwise we will get the value 380 instead of "Gd"

Shown below

In [ ]:
# here we can also use mode to find the feature with most frequency
# Both shown method gives same result
# we can use anyone as desired
df["FireplaceQu"].mode()[0]
df["FireplaceQu"].value_counts().index[0]

'Gd'

Now writing a function to replace all The NAN values in the specific features with most occured categorical feature

In [ ]:
def impute_nan(df,variable):
  most_frequent_category=df[variable].value_counts().index[0]
  df[variable].fillna(most_frequent_category,inplace=True)

In [ ]:
# we may have done one by one for each feature but lets do it together for all 3 features using for loop
for feature in ["BsmtQual","FireplaceQu","GarageType",]:
  impute_nan(df,feature)
df.head(5)

,BsmtQual,FireplaceQu,GarageType,SalePrice
0,Gd,Gd,Attchd,208500
1,Gd,TA,Attchd,181500
2,Gd,TA,Attchd,223500
3,TA,Gd,Detchd,140000
4,Gd,TA,Attchd,250000


In [ ]:
df.isnull().mean()

BsmtQual       0.0
FireplaceQu    0.0
GarageType     0.0
SalePrice      0.0
dtype: float64

Advantages and disadvantages of using : Frequent category Imputation

Advantages 
1) Easy to implement 
2) faster way to implement 

Diadvantages
1) Since we are using the most frequent labels, it may use them in an over represented way if there are many NAN
2) It distorts the relation of the most frequent label

Technique 2 : Adding a variable to capture NAN

This also works for those features which has high number of NAN values too

In [ ]:
df=pd.read_csv("train_house.csv",usecols=["BsmtQual","FireplaceQu","GarageType","SalePrice"])
df.head(4)

,BsmtQual,FireplaceQu,GarageType,SalePrice
0,Gd,NaN,Attchd,208500
1,Gd,TA,Attchd,181500
2,Gd,TA,Attchd,223500
3,TA,Gd,Detchd,140000


In [ ]:
import numpy as np
df["BsmtQual_Var"]=np.where(df["BsmtQual"].isnull(),1,0)
df.head()

,BsmtQual,FireplaceQu,GarageType,SalePrice,BsmtQual_Var
0,Gd,NaN,Attchd,208500,0
1,Gd,TA,Attchd,181500,0
2,Gd,TA,Attchd,223500,0
3,TA,Gd,Detchd,140000,0
4,Gd,TA,Attchd,250000,0


Here we have shown the importance of the values whether they are NAN or not in BsmtQual_Var

Now replacing the NAN in BsmtQual using the most frequent category method itself.

In [ ]:
most_frequent=df["BsmtQual"].mode()[0]

In [ ]:
df["BsmtQual"].fillna(most_frequent,inplace=True)
df.head()

,BsmtQual,FireplaceQu,GarageType,SalePrice,BsmtQual_Var
0,Gd,NaN,Attchd,208500,0
1,Gd,TA,Attchd,181500,0
2,Gd,TA,Attchd,223500,0
3,TA,Gd,Detchd,140000,0
4,Gd,TA,Attchd,250000,0


In [ ]:
# This method also works for features which have the high number of NAN values(Here FireplaceQu has the highest number of NAN values) 
#,the previous technique couldnt handle this

df["FireplaceQu_Var"]=np.where(df["FireplaceQu"].isnull(),1,0)
most_frequent=df["FireplaceQu"].mode()[0]
df["FireplaceQu"].fillna(most_frequent,inplace=True)
df.head()


,BsmtQual,FireplaceQu,GarageType,SalePrice,BsmtQual_Var,FireplaceQu_Var
0,Gd,Gd,Attchd,208500,0,1
1,Gd,TA,Attchd,181500,0,0
2,Gd,TA,Attchd,223500,0,0
3,TA,Gd,Detchd,140000,0,0
4,Gd,TA,Attchd,250000,0,0


Sub technique : this technique is most frequently used

**Replacing NAN with a new category **

In [ ]:
df=pd.read_csv("train_house.csv",usecols=["BsmtQual","FireplaceQu","GarageType","SalePrice"])
df.head()

,BsmtQual,FireplaceQu,GarageType,SalePrice
0,Gd,NaN,Attchd,208500
1,Gd,TA,Attchd,181500
2,Gd,TA,Attchd,223500
3,TA,Gd,Detchd,140000
4,Gd,TA,Attchd,250000


In [ ]:
import numpy as np
def impute_nan(df,variable):
  df[variable+"newvar"]=np.where(df[variable].isnull(),"Missing",df[variable])


In [ ]:
for feature in ["BsmtQual","FireplaceQu","GarageType",]:
  impute_nan(df,feature)
df.head(5)

,BsmtQual,FireplaceQu,GarageType,SalePrice,BsmtQualnewvar,FireplaceQunewvar,GarageTypenewvar
0,Gd,NaN,Attchd,208500,Gd,Missing,Attchd
1,Gd,TA,Attchd,181500,Gd,TA,Attchd
2,Gd,TA,Attchd,223500,Gd,TA,Attchd
3,TA,Gd,Detchd,140000,TA,Gd,Detchd
4,Gd,TA,Attchd,250000,Gd,TA,Attchd


In [ ]:
# As now we have created new faetures with no NAN values we can drop the Fetaures which had NAN values
df=df.drop(["BsmtQual","FireplaceQu","GarageType",],axis=1)
df.head()

,SalePrice,BsmtQualnewvar,FireplaceQunewvar,GarageTypenewvar
0,208500,Gd,Missing,Attchd
1,181500,Gd,TA,Attchd
2,223500,Gd,TA,Attchd
3,140000,TA,Gd,Detchd
4,250000,Gd,TA,Attchd
